## 循环神经网络的程序示例

### 1. RNN 架构

### 2. 实现循环神经网络

- 利用tensorflow实现的循环神经网络RNN（本程序使用了LSTM）来做语言模型，并输出其困惑度。
- 语言模型主要是根据一段给定的文本来预测下一个词最有可能是什么。困惑度用于评价语言模型。困惑度越小，则模型的性能越好

In [2]:
import reader
import numpy as np
import tensorflow as tf

print("tensorflow version : ", tf.__version__)

tensorflow version :  2.3.0


#### 2.1 定义参数

- data目录中应该预先存放本程序中要用到的语料，即PTB(Penn Treebank Dataset)数据集
- PTB数据集是语言模型研究常用的数据集。其下载地址在 Tomas Mikolov的主页：
- 下载地址 : https://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz。 (推荐使用迅雷下载)
- 解压后，将其中的整个data文件夹拷贝到当前目录即可。数据集共有9998个单词，加上稀有词语的特殊符号<unk>和语句的结束标记，共有10000个单词。

In [3]:
# 数据路径
DATA_PATH = './data'

# 超参数的设置
# size of RNN hidden state, 每个单词词向量的维度
HIDDEN_SIZE = 200
# LSTM 层数
NUM_LAYERS = 2
# 词汇表中词的个数
VOCAB_SIZE = 10000
# 学习速率的超参数
LEARNING_RATE = 1.0
# 训练阶段每个数据批量设置为多少个样本, 本例中指若干个词构成的序列
TRAIN_BATCH_SIZE = 20
# 训练阶段文本数据的截断长度, 也可以成为序列长度 seq_length
TRAIN_NUM_STEP = 35
# 训练的轮数
NUM_EPOCH = 2
# 节点不被 dropout 的概率
KEEP_PROB = 0.5

# 超参数, 避免梯度膨胀
MAX_GRAD_NORM = 5

### 2.2 定义多层循环神经网络

In [ ]:
class PTBModel(object):
    def __init__(self, is_training, batch_size, num_steps):
        """
        初始化参数
        :param is_training: 当前是否在训练阶段
        :param batch_size: 批量的大小
        :param num_steps: 数据的截断长度, 即序列长度
        """
        # 定义输入层的数据维度为 batch_size * num_steps
        self.input_data = tf.keras.Input(dtype=tf.int32, shape=[batch_size, num_steps])
        # 定义输出层的数据维度为 batch_size * num_steps
        self.targets = tf.keras.Input(dtype=tf.int32, shape=[batch_size, num_steps])

        # 定义使用LSTM机构作为循环体的基本结构, 每个词向量的维度为 HIDDEN_SIZE
        lstm_cell = tf.compat.v1.nn.rnn_cell.LSTMCell()

        #如果是在训练阶段，则使用dropout.此时每个单元以（1-keep_prob）的概率不工作，目的是防止过拟合。
        if is_training:
            lstm_cell = tf.compat.v1.nn.rnn_cell.DropoutWrapper(lstm_cell, output_keep_prob=KEEP_PROB)

        # 将多层RNN单元封装到一个单元Cell中, 层的个数NUM_LAYERS前面已经确定
        cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell([lstm_cell]*NUM_LAYERS)

        # 使用zero_state函数初始化网络状态
        self.initial_state = cell.zero_state(batch_size, tf.int32)

        # 将单词转换为词向量
        # VOCAB_SIZE 词的总数, HIDDEN_SIZE 是每个单词的向量维度
        # embedding 的参数为 : VOCAB_SIZE * HIDDEN_SIZE
        # 则input的输入维度为 : batch_size * num_steps * HIDDEN_SIZE
        embedding = tf.compat.v1.get_variable()